# **`Hodvats`** **IA**

---



---



### `Instalação de Pacotes (Biblioteca)`

In [ ]:
!pip install -U -q google-generativeai
!pip install reportlab

In [ ]:
!pip install markdown2

### `Importação das Bibliotecas`

In [ ]:
import google.generativeai as genai
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet

In [ ]:
import markdown2
import textwrap
from IPython.display import display
from IPython.display import Markdown

# **`Congiguração do Modelo de IA`**

In [ ]:
Google_API_KEY='YOUR_API_KEY'
# Configure a biblioteca cliente fornecendo sua chave API.
genai.configure(api_key=Google_API_KEY)

# Configure o modelo
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)



### **`Crindo um Artigo`**


In [ ]:
chat = model.start_chat(history=[])

prompt = input("Esperando prompt: ")

while prompt != "fim":
  response = chat.send_message(prompt)
  print(response.text)
  prompt = input("Esperando prompt: ")

### `Criar o nome e o Tema do Artigo `

In [ ]:
nome_artigo = input("Digite o nome do arquivo: ")
tem_artigo = input("Digite o Tema do arquivo: ")

# **`Gerar o Artigo em PDF`**

In [ ]:
# Define uma função `to_markdown()` que converte texto em formato Markdown
def to_markdown(text):
    # Substitui todos os pontos (`.`) por asteriscos (`*`)
    text = text.replace('.', '*')

    # Recua o texto com três espaços
    return Markdown(textwrap.indent(text, '>', predicate=lambda x: True))

# Imprime o histórico da conversa
for message in chat.history:
    # Converte o texto da mensagem em formato Markdown
    formatted_text = to_markdown(f'**{message.role}**: {message.parts[0].text}')


In [ ]:
def crar_pdf(noticias):
    doc = SimpleDocTemplate(f"{nome_artigo}.pdf", pagesize=letter)
    styles = getSampleStyleSheet()

    # Lista para armazenar o conteúdo das notícias
    conteudo = []

    # Adiciona um cabeçalho ao documento
    cabeçalho = Paragraph(f"{tem_artigo}", styles["Title"])
    conteudo.append(cabeçalho)
    conteudo.append(Spacer(1, 12))

    # Adiciona o histórico da conversa ao documento
    conteudo.append(Paragraph("Artigo:", styles["Heading2"]))
    for mensagem in chat.history:
        texto_formatado = markdown2.markdown(f"**{mensagem.role}**: {mensagem.parts[0].text}")
        conteudo.append(Paragraph(texto_formatado, styles["Normal"]))
        conteudo.append(Spacer(1, 6))  # Espaçamento menor entre mensagens

    # Adiciona as notícias ao documento (se houver)
    for i, noticia in enumerate(noticias, start=1):
        paragrafo = Paragraph(f"<b>Artigo {i}:</b> {noticia}", styles["Normal"])
        conteudo.append(paragrafo)
        conteudo.append(Spacer(1, 12))

    doc.build(conteudo)

 # Exemplo de utilização (mantendo o mesmo código)
def main():
    num_noticias = 1  # Defina o número de notícias desejado
    noticias = []
    for i in range(num_noticias):
        noticia = formatted_text,  # Substitua por sua fonte de notícias
        noticias.append(noticia)

    crar_pdf(noticias)
    print("PDF gerado com sucesso!")

if __name__ == "__main__":
    main()